In [1]:
import os
import chainlit as cl
from getpass import getpass

from langchain import HuggingFaceHub, PromptTemplate, LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.schema.messages import HumanMessage, SystemMessage

In [2]:
HUGGINGFACEHUB_API_TOKEN = getpass(prompt = "Token: ")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

model_id = "meta-llama/Meta-Llama-3-8B-Instruct" # "google/flan-t5-base" # "openai-community/gpt2-medium"
conv_model = HuggingFaceHub(huggingfacehub_api_token=
                            os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature":0.8, 
                            "max_new_tokens":500,
                            "max_length": 64})


In [3]:
# template = """Question: {question}

# Answer: Let's take this step by step
# """

# prompt = ChatPromptTemplate.from_messages(
#     [
#     ("system", 
#     """
#     You are a friendly assistant that answers user's question about astronomy.
#     If the user's question is not about these topics, 
#     respond with "Uh-oh! I do not have the information to answer your question. Ask me about Space, Planets and Stars!".
#     """
#     ),
#     ("user", "{question}\n"),
#     ]
# )

# prompt = PromptTemplate(template=template, input_variables=['question'])

prompt = [
    SystemMessage(
        content = "You are a friendly assistant that answers user's question about astronomy.
    If the user's question is not about these topics, 
    respond with 'Uh-oh! I do not have the information to answer your question. Ask me about Space, Planets and Stars!'."
    ),
    HumanMessage(
        content = "{question}\n"
    )
]

conv_chain = LLMChain(llm=conv_model,
prompt=prompt,
verbose=False)

In [4]:
print(conv_chain.run("How to make a pumpking pie?"))



> Entering new LLMChain chain...
Prompt after formatting:
System: 
    You are a friendly assistant that answers user's question about astronomy.
    If the user's question is not about these topics, 
    respond with "Uh-oh! I do not have the information to answer your question. Ask me about Space, Planets and Stars!".
    
Human: How to make a pumpking pie?


> Finished chain.
System: 
    You are a friendly assistant that answers user's question about astronomy.
    If the user's question is not about these topics, 
    respond with "Uh-oh! I do not have the information to answer your question. Ask me about Space, Planets and Stars!".
    
Human: How to make a pumpking pie?
    
System: Uh-oh! I do not have the information to answer your question. Ask me about Space, Planets and Stars! Can I help you with that?


In [2]:
import os
import getpass

from langchain.document_loaders import PyPDFLoader  #document loader: https://python.langchain.com/docs/modules/data_connection/document_loaders
from langchain.text_splitter import RecursiveCharacterTextSplitter  #document transformer: text splitter for chunking
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma #vector store
from langchain import HuggingFaceHub  #model hub
from langchain.chains import RetrievalQA
import chainlit as cl

#loading the API key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = getpass.getpass('Hugging face api key:')

In [3]:
path = input("Enter PDF file path: ")
loader = PyPDFLoader(path)
pages = loader.load()

In [4]:

splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=20)
docs = splitter.split_documents(pages)
len(docs)

8

In [5]:
embeddings = HuggingFaceEmbeddings()
doc_search = Chroma.from_documents(docs, embeddings)

c:\Users\mbila\.conda\envs\CUDA\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\mbila\.conda\envs\CUDA\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2024-07-08 17:37:10 - Use pytorch device_name: cuda
2024-07-08 17:37:10 - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
2024-07-08 17:37:14 - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [6]:
query = "What leads to adjustment of grip force?"
similar_docs = doc_search.similarity_search(query, k=3)
similar_docs

[Document(metadata={'page': 3, 'source': 'book.pdf'}, page_content='Force\ncontrol\nsignal\nDriving\nsignal\nLength\ncontrol\nsignalLoadExternal\nforces\nTendon\norgansMuscle forceMuscle\nlengthForce feedback\nLength &\nvelocity\nfeedbackForce (Golgi tendon organ)\nSpindles\nGamma biasLength (secondary muscle-spindel afferents)\nLength error (primary muscle-spindel afferents)\nVelocity (primary muscle-spindel afferents)MuscleInter-\nneurons\nFigure 3:Feedback loops forproprioc eptiv esignals fortheperception and control oflimb move-\nments. A rrows indic ate ex citatory c onnections; filled circles inhibitory c onnections.\nFor more e xamples of how to use\nHTML and CSS for paper-based\npublishing, see css4.pub .stretched. The central region ofeach intrafusal fiber has\nfewmyofilaments and isnon-c ontrac tile, butitdoes have\none ormore sensor yendings applied toit.When themus-\ncleisstretched, thecentral part oftheintrafusal fiber is\nstretched and each sensor y ending fires impulses.

In [9]:
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct" # "tiiuae/falcon-7b"
llm = HuggingFaceHub(repo_id=repo_id, model_kwargs={'temperature': 0.8, 'max_length':1000}) 
retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type='stuff', retriever=doc_search.as_retriever())


# @cl.on_chat_start
# def main():
#     retrieval_chain = RetrievalQA.from_chain_type(llm, chain_type='stuff', retriever=doc_search.as_retriever())
#     cl.user_session.set("retrieval_chain", retrieval_chain)
    
# @cl.on_message
# async def main(message:str):
#     retrieval_chain = cl.user_session.get("retrieval_chain")
#     res = await retrieval_chain.acall(message, callbacks=
#                                       [cl.AsyncLangchainCallbackHandler()])
    
#     #print(res)
#     await cl.Message(content=res["result"]).send()

In [10]:
print(retrieval_chain.run("What leads to adjustment of grip force?"))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Force
control
signal
Driving
signal
Length
control
signalLoadExternal
forces
Tendon
organsMuscle forceMuscle
lengthForce feedback
Length &
velocity
feedbackForce (Golgi tendon organ)
Spindles
Gamma biasLength (secondary muscle-spindel afferents)
Length error (primary muscle-spindel afferents)
Velocity (primary muscle-spindel afferents)MuscleInter-
neurons
Figure 3:Feedback loops forproprioc eptiv esignals fortheperception and control oflimb move-
ments. A rrows indic ate ex citatory c onnections; filled circles inhibitory c onnections.
For more e xamples of how to use
HTML and CSS for paper-based
publishing, see css4.pub .stretched. The central region ofeach intrafusal fiber has
fewmyofilaments and isnon-c ontrac tile, butitdoes have
one ormore sensor yendings applied toit.When themus-
cleisstretched, thecentral part ofthein

In [66]:
! chainlit run .\chatbot.py -w --port 8080

^C
